# 001 - maps

This notebook loads all the .csv files in a directory, and plots the GPS coordinates on a map.

In [ ]:
from pyprojroot import here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import folium

# Define file locations

In [ ]:
project_root = here()
csv_dir = project_root / 'data' / 'greece' / 'csv'
save_dir = project_root / 'figures'

In [ ]:
map_name = 'greece'

# Load csv files

In [ ]:
# All items in csv_dir should be .csv files.
csv_file_paths = [csv_file_path for csv_file_path in csv_dir.iterdir()]

In [ ]:
list_of_records = []
for csv_file_path in tqdm(csv_file_paths):
    # Row 0 contains the variable names.
    # Row 1 contains the variable units.
    # Column 0 contains the timestamp, which should be used as the index.
    records = pd.read_csv(csv_file_path, header=[0, 1], index_col=0)
    records.index = pd.to_datetime(records.index)
    
    # Drop all rows without GSP coordinates
    records = records.dropna(subset=[('position_long', 'semicircles'), ('position_lat', 'semicircles')])
    
    # Convert the GSP coordinates from semicircles to degrees
    records.loc[:, ('position_long', 'deg')] = records['position_long']['semicircles']*180/2**31
    records.loc[:, ('position_lat', 'deg')] = records['position_lat']['semicircles']*180/2**31
    
    list_of_records.append(records)

print('Timestamp (first) : ', list_of_records[ 0].index[ 0])
print('Timestamp (last)  : ', list_of_records[-1].index[-1])

# Correct the timestamps for a different timezone

This should only be done once because modifying the `records` in the `for` loop modifies the `list_of_records` element permanently.

In [ ]:
timezone = pd.Timedelta(hours=2)

In [ ]:
print('Timestamp (original)  : ', list_of_records[ 0].index[ 0])
for records in list_of_records:
    records.index = records.index + timezone
print('Timestamp (corrected) : ', list_of_records[ 0].index[ 0])

# Plot the GPS coordinates of all records

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[16, 20])
for records in list_of_records:
    ax.plot(
        records['position_long']['deg'],
        records['position_lat']['deg'],
        color='k',
    )
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('equal')
    ax.axis('off')

# Save figure as vector svg file
fig.savefig(save_dir / (map_name + '.svg'), format='svg', transparent=True, bbox_inches=0)

In [ ]:
# Calculate the map centre from the average coordinates of all records
records_centres = [
    [records['position_lat']['deg'].mean(), records['position_long']['deg'].mean()] 
    for records in list_of_records]
map_centre = (np.array(records_centres).min(axis=0)+np.array(records_centres).max(axis=0))/2

# Build map 
folium_map = folium.Map(
    location=map_centre, 
    zoom_start=9, 
    tiles='OpenStreetMap',
    width=900, 
    height=600,
)

# Add GPS coordinates to map
for records in list_of_records:
    folium.ColorLine(
        records.loc[:, [('position_lat', 'deg'), ('position_long', 'deg')]].values,
        colors=range(len(records)),
        colormap=['black', 'black'],
        fill_color='#0080bb',
    ).add_to(folium_map)

# Save map to html file
folium_map.save(str(save_dir / f'{map_name}.html'))

# Display map in Jupyter
folium_map

# Plot the GPS coordinates of a given date

In [ ]:
date = '2022-04-20'

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[16, 20])
for records in list_of_records:
    ax.plot(
        records['position_long']['deg'],
        records['position_lat']['deg'],
        color='k',
    )
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('equal')

for records in list_of_records:
    records = records.loc[date:date].copy()
    ax.plot(
        records['position_long']['deg'],
        records['position_lat']['deg'],
        color='deepskyblue',
        linewidth=5,
    )
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('equal')
    ax.axis('off')

# Save figure as vector svg file
fig.savefig(save_dir / ('%s-%s.svg' % (map_name, date)), format='svg', transparent=True, bbox_inches=0)